Plus court chemin dans une matrice
==================================

On considère la matrice $M=\left(\begin{array}{rrrrr}131&673&234&103&18\\201&96&342&965&150\\630&803&746&422&111\\537&699&497&121&956\\805&732&524&37&331\\\end{array}\right)$ et on cherche le plus court chemin en partant de la case $(1,1)$ jusqu'à la case $(5,5)$, avec des déplacements uniquement vers le sud et vers l'est, la longueur d'un chemin étant la somme des coefficients rencontrés le long du chemin.

Par exemple le chemin en rouge a comme longueur (ou poids) $131+201+96+342+746+422+121+37+331=2427$.

<table>
<tr><td><span style="color:red">131</span></td><td>673</td><td>234</td><td>103</td><td>18</td></tr>
<tr><td><span style="color:red">201</span></td><td><span style="color:red">96</span></td><td><span style="color:red">342</span></td><td>965</td><td>150</td></tr>
<tr><td>630</td><td>803</td><td><span style="color:red">746</span></td><td><span style="color:red">422</span></td><td>111</td></tr>
<tr><td>537</td><td>699</td><td>497</td><td><span style="color:red">121</span></td><td>956</td></tr>
<tr><td>805</td><td>732</td><td>524</td><td><span style="color:red">37</span></td><td><span style="color:red">331</span></td></tr>
</table>

On représente cette matrice par une **grille**, un tableau de tableaux de même longueur.

Attention, les indices mathématiques commencent ici à 1 tandis que les indices informatiques commencent à zéro. 

P.ex le 96 qui est sur la diagonale est noté $m_{2,2}$ en Maths et `T[1][1]` en Info, le 630 est $m_{3,1}$ ou `T[2][0]`.

In [56]:
T=[[131,673,234,103,18],
   [201,96,342,965,150],
   [630,803,746,422,111],
   [537,699,497,121,956],
   [805,732,524,37,331]]

## Une approche récursive

On utilise la relation de récurrence $d_{i,j}=\left\{\begin{array}{lll}m_{1,1}&\text{ si }& i=1 \text{ et } j=1\\d_{i-1,j}+m_{i,j}&\text{ si }& i>1\text{ et }j=1\\d_{i,j-1}+m_{i,j}&\text{ si }&i=1 \text{ et } j>1\\min\left(d_{i-1,j},d_{i,j-1}\right)+m_{i,j}&\text{ si }&i>1 \text{ et } j>1\\\end{array}\right.$

qui se traduit directement en une fonction récursive.

In [57]:
def distance(T,i,j):
    if i==0 and j==0:
        return T[0][0]
    elif i>0 and j==0:
        return T[i][j]+distance(T,i-1,j)
    elif i==0 and j>0:
        return T[i][j]+distance(T,i,j-1)
    else:
        return T[i][j]+min(distance(T,i-1,j),distance(T,i,j-1))

In [58]:
distance(T,4,4)

2427

## Mémoïsation 
Un `dict` permet de mémoriser les valeurs déjà calculées, la relation de récurrence reste la même.

In [59]:
def distance_memo(T,i,j,d={}):
    if (i,j) in d:
        return d[(i,j)]
    if i==0 and j==0:
        m=T[0][0]
    elif i>0 and j==0:
        m=T[i][j]+distance_memo(T,i-1,j,d)
    elif i==0 and j>0:
        m=T[i][j]+distance_memo(T,i,j-1,d)
    else:
        m=T[i][j]+min(distance_memo(T,i-1,j,d),distance_memo(T,i,j-1,d))
    d[(i,j)]=m
    return m

In [60]:
distance_memo(T,4,4)

2427

## Dérécursification : une version itérative

L'idée est de construire un tableau `U` contenant le plus court chemin vers chacune des cases, encore avec la même relation de récurrence, mais dans une double boucle.

In [61]:
def distance_prog_dyn(T,i,j):
    U=[[0]*(j+1) for u in range(i+1)]
    U[0][0]=T[0][0]
    for u in range(1,i+1):
        U[u][0]=U[u-1][0]+T[u][0]
    for v in range(1,j+1):
        U[0][v]=U[0][v-1]+T[0][v]
    for u in range(1,i+1):
        for v in range(1,j+1):
            U[u][v]=T[u][v]+min(U[u-1][v],U[u][v-1])
    print(U)
    return U[i][j]

In [62]:
distance_prog_dyn(T,4,4)

[[131, 804, 1038, 1141, 1159], [332, 428, 770, 1735, 1309], [962, 1231, 1516, 1938, 1420], [1499, 1930, 2013, 2059, 2376], [2304, 2662, 2537, 2096, 2427]]


2427

**Attention** pour une grille, `U=[[0]*3]*4` ne donnera pas le résultat voulu car toutes les lignes sont le même objet.

In [63]:
U=[[0]*3]*4
U

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [64]:
U[1][2]=42
U

[[0, 0, 42], [0, 0, 42], [0, 0, 42], [0, 0, 42]]

## La bibiliothèque pretty print

On veut un affichage plus joli pour nos grilles.

In [65]:
from pprint import pprint
pprint(T)

[[131, 673, 234, 103, 18],
 [201, 96, 342, 965, 150],
 [630, 803, 746, 422, 111],
 [537, 699, 497, 121, 956],
 [805, 732, 524, 37, 331]]


## Retrouver le chemin

On modifie la fonction `distance_prog_dyn` pour renvoyer aussi la grille. Grâce à ce tableau, on peut alors retrouver le plus court chemin sous la forme d'une liste de couples ; on empile les positions des cases parcourues en partant de la dernière, puis on renvoie la liste dans le bon ordre.

In [66]:
def distance_prog_dyn(T,i,j):
    U=[[0]*(j+1) for u in range(i+1)]
    U[0][0]=T[0][0]
    for u in range(1,i+1):
        U[u][0]=U[u-1][0]+T[u][0]
    for v in range(1,j+1):
        U[0][v]=U[0][v-1]+T[0][v]
    for u in range(1,i+1):
        for v in range(1,j+1):
            U[u][v]=T[u][v]+min(U[u-1][v],U[u][v-1])
    return U[i][j],U

In [67]:
distance_prog_dyn(T,4,4)

(2427, [[131, 804, 1038, 1141, 1159], [332, 428, 770, 1735, 1309], [962, 1231, 1516, 1938, 1420], [1499, 1930, 2013, 2059, 2376], [2304, 2662, 2537, 2096, 2427]])

In [71]:
def chemin(T,U,i,j):
    u,v=i,j
    nimehc=[]
    nimehc.append((u,v))
    while u>0 or v>0:
        if u==0:
            v=v-1
        elif v==0:
            u=u-1
        elif U[u-1][v]<U[u][v-1]:
            u=u-1
        else:
            v=v-1
        nimehc.append((u,v))
    chemin=[]
    while nimehc!=[]:
        chemin.append(nimehc.pop())
    return chemin

In [72]:
l,U=distance_prog_dyn(T,4,4)

In [73]:
chemin(T,U,4,4)

[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (3, 3), (4, 3), (4, 4)]

## Mesurer le temps d'exécution

Pour chronométrer le temps d'exécution d'un fragment de code on peut utiliser la fonction `time` du module `time` comme dans l'exemple ci-dessous.  
**The Epoch** est le premier janvier 1970 à 0h00:00, début de l'ère d'Unix.

In [74]:
from time import time
help(time)

Help on built-in function time in module time:

time(...)
    time() -> floating point number
    
    Return the current time in seconds since the Epoch.
    Fractions of a second may be present if the system clock provides them.



In [75]:
t0=time()
s=0
for k in range(10**5):
    s=s+k
t1=time()
print(t1-t0)

0.05299997329711914


## Visualiser les détails avec PythonTutor

In [76]:
from tutor import tutor

def distance_prog_dyn(T,i,j):
    U=[[0]*(j+1) for u in range(i+1)]
    U[0][0]=T[0][0]
    for u in range(1,i+1):
        U[u][0]=U[u-1][0]+T[u][0]
    for v in range(1,j+1):
        U[0][v]=U[0][v-1]+T[0][v]
    for u in range(1,i+1):
        for v in range(1,j+1):
            U[u][v]=T[u][v]+min(U[u-1][v],U[u][v-1])
    return U[i][j],U

T=[[131,673,234,103,18],
   [201,96,342,965,150],
   [630,803,746,422,111],
   [537,699,497,121,956],
   [805,732,524,37,331]]

l,U=distance_prog_dyn(T,4,4)

tutor()

<iframe id="basthon-pythontutor-iframe-2" style="width: 100%; height: 400px;" frameborder="0" src="data:text/html;charset=utf-8,%3C%21DOCTYPE%20html%3E%0A%3Chtml%3E%0A%20%20%3Chead%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//notebook.basthon.fr/assets/0.62.14/python3/modules/extern/pytutor-main.min.css%22/%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//notebook.basthon.fr/assets/0.62.14/python3/modules/extern/pytutor-main.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%3E%0A%20%20%20%20%20%20let%20sent_height%3B%0A%20%20%20%20%20%20%24%28document%29.ready%28function%28%29%20%7B%0A%20%20%20%20%20%20%20%20%20%20//%20managing%20iframe%20resize%0A%20%20%20%20%20%20%20%20%20%20const%20send%20%3D%20%28%29%20%3D%3E%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20const%20new_height%20%3D%20document.body.offsetHeight%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20if%20%28sent_height%20%3D%3D%3D%20new_height%29%20return%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20sent_height%20%3D%20new_height%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20window.parent.postMessage%28%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20type%3A%20%22pytutor-iframe-resize%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20target%3A%20%22basthon-pythontutor-iframe-2%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20sent_height%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%2C%20%27%2A%27%29%3B%0A%20%20%20%20%20%20%20%20%20%20%7D%3B%0A%20%20%20%20%20%20%20%20%20%20const%20o%20%3D%20new%20ResizeObserver%28send%29%3B%0A%20%20%20%20%20%20%20%20%20%20o.observe%28document.body%29%3B%0A%0A%20%20%20%20%20%20%20%20%20%20const%20t%20%3D%20%7B%22code%22%3A%20%22def%20distance_prog_dyn%28T%2Ci%2Cj%29%3A%5Cn%20%20%20%20U%3D%5B%5B0%5D%2A%28j%2B1%29%20for%20u%20in%20range%28i%2B1%29%5D%5Cn%20%20%20%20U%5B0%5D%5B0%5D%3DT%5B0%5D%5B0%5D%5Cn%20%20%20%20for%20u%20in%20range%281%2Ci%2B1%29%3A%5Cn%20%20%20%20%20%20%20%20U%5Bu%5D%5B0%5D%3DU%5Bu-1%5D%5B0%5D%2BT%5Bu%5D%5B0%5D%5Cn%20%20%20%20for%20v%20in%20range%281%2Cj%2B1%29%3A%5Cn%20%20%20%20%20%20%20%20U%5B0%5D%5Bv%5D%3DU%5B0%5D%5Bv-1%5D%2BT%5B0%5D%5Bv%5D%5Cn%20%20%20%20for%20u%20in%20range%281%2Ci%2B1%29%3A%5Cn%20%20%20%20%20%20%20%20for%20v%20in%20range%281%2Cj%2B1%29%3A%5Cn%20%20%20%20%20%20%20%20%20%20%20%20U%5Bu%5D%5Bv%5D%3DT%5Bu%5D%5Bv%5D%2Bmin%28U%5Bu-1%5D%5Bv%5D%2CU%5Bu%5D%5Bv-1%5D%29%5Cn%20%20%20%20return%20U%5Bi%5D%5Bj%5D%2CU%5Cn%5CnT%3D%5B%5B131%2C673%2C234%2C103%2C18%5D%2C%5Cn%20%20%20%5B201%2C96%2C342%2C965%2C150%5D%2C%5Cn%20%20%20%5B630%2C803%2C746%2C422%2C111%5D%2C%5Cn%20%20%20%5B537%2C699%2C497%2C121%2C956%5D%2C%5Cn%20%20%20%5B805%2C732%2C524%2C37%2C331%5D%5D%5Cn%5Cnl%2CU%3Ddistance_prog_dyn%28T%2C4%2C4%29%22%2C%20%22trace%22%3A%20%5B%7B%22line%22%3A%201%2C%20%22event%22%3A%20%22step_line%22%2C%20%22func_name%22%3A%20%22%3Cmodule%3E%22%2C%20%22globals%22%3A%20%7B%7D%2C%20%22ordered_globals%22%3A%20%5B%5D%2C%20%22stack_to_render%22%3A%20%5B%5D%2C%20%22heap%22%3A%20%7B%7D%2C%20%22stdout%22%3A%20%22%22%7D%2C%20%7B%22line%22%3A%2013%2C%20%22event%22%3A%20%22step_line%22%2C%20%22func_name%22%3A%20%22%3Cmodule%3E%22%2C%20%22globals%22%3A%20%7B%22distance_prog_dyn%22%3A%20%5B%22REF%22%2C%201%5D%7D%2C%20%22ordered_globals%22%3A%20%5B%22distance_prog_dyn%22%5D%2C%20%22stack_to_render%22%3A%20%5B%5D%2C%20%22heap%22%3A%20%7B%221%22%3A%20%5B%22FUNCTION%22%2C%20%22distance_prog_dyn%28T%2C%20i%2C%20j%29%22%2C%20null%5D%7D%2C%20%22stdout%22%3A%20%22%22%7D%2C%20%7B%22line%22%3A%2014%2C%20%22event%22%3A%20%22step_line%22%2C%20%22func_name%22%3A%20%22%3Cmodule%3E%22%2C%20%22globals%22%3A%20%7B%22distance_prog_dyn%22%3A%20%5B%22REF%22%2C%201%5D%7D%2C%20%22ordered_globals%22%3A%20%5B%22distance_prog_dyn%22%5D%2C%20%22stack_to_render%22%3A%20%5B%5D%2C%20%22heap%22%3A%20%7B%221%22%3A%20%5B%22FUNCTION%22%2C%20%22distance_prog_dyn%28T%2C%20i%2C%20j%29%22%2C%20null%5D%7D%2C%20%22stdout%22%3A%20%22%22%7D%2C%20%7B%22line%22%3A%2015%2C%20%22event%22%3A%20%22s

## Charger un fichier

On va écrire une fonction `lire(nom)` qui lit un fichier dont le nom est passé en argument et qui renvoie une grille d'entiers.

Le fichier `data.txt` contient une instance $80\times 80$.

In [77]:
def lire(nom):
    tab=[]
    with open(nom,'r') as f:
        for ligne in f:
            tab.append([int(k) for k in ligne.strip().split(',')])
    return tab

In [78]:
T=lire('p081_matrix.txt')

In [79]:
T

[[4445, 2697, 5115, 718, 2209, 2212, 654, 4348, 3079, 6821, 7668, 3276, 8874, 4190, 3785, 2752, 9473, 7817, 9137, 496, 7338, 3434, 7152, 4355, 4552, 7917, 7827, 2460, 2350, 691, 3514, 5880, 3145, 7633, 7199, 3783, 5066, 7487, 3285, 1084, 8985, 760, 872, 8609, 8051, 1134, 9536, 5750, 9716, 9371, 7619, 5617, 275, 9721, 2997, 2698, 1887, 8825, 6372, 3014, 2113, 7122, 7050, 6775, 5948, 2758, 1219, 3539, 348, 7989, 2735, 9862, 1263, 8089, 6401, 9462, 3168, 2758, 3748, 5870], [1096, 20, 1318, 7586, 5167, 2642, 1443, 5741, 7621, 7030, 5526, 4244, 2348, 4641, 9827, 2448, 6918, 5883, 3737, 300, 7116, 6531, 567, 5997, 3971, 6623, 820, 6148, 3287, 1874, 7981, 8424, 7672, 7575, 6797, 6717, 1078, 5008, 4051, 8795, 5820, 346, 1851, 6463, 2117, 6058, 3407, 8211, 117, 4822, 1317, 4377, 4434, 5925, 8341, 4800, 1175, 4173, 690, 8978, 7470, 1295, 3799, 8724, 3509, 9849, 618, 3320, 7068, 9633, 2384, 7175, 544, 6583, 1908, 9983, 481, 4187, 9353, 9377], [9607, 7385, 521, 6084, 1364, 8983, 7623, 1585, 6935, 

In [80]:
distance_prog_dyn(T,79,79)

(427337, [[4445, 7142, 12257, 12975, 15184, 17396, 18050, 22398, 25477, 32298, 39966, 43242, 52116, 56306, 60091, 62843, 72316, 80133, 89270, 89766, 97104, 100538, 107690, 112045, 116597, 124514, 132341, 134801, 137151, 137842, 141356, 147236, 150381, 158014, 165213, 168996, 174062, 181549, 184834, 185918, 194903, 195663, 196535, 205144, 213195, 214329, 223865, 229615, 239331, 248702, 256321, 261938, 262213, 271934, 274931, 277629, 279516, 288341, 294713, 297727, 299840, 306962, 314012, 320787, 326735, 329493, 330712, 334251, 334599, 342588, 345323, 355185, 356448, 364537, 370938, 380400, 383568, 386326, 390074, 395944], [5541, 5561, 6879, 14465, 19632, 20038, 19493, 25234, 32855, 39328, 44854, 47486, 49834, 54475, 64302, 65291, 72209, 78092, 81829, 82129, 89245, 95776, 96343, 102340, 106311, 112934, 113754, 119902, 123189, 125063, 133044, 141468, 149140, 156715, 163512, 170229, 171307, 176315, 180366, 189161, 194981, 195327, 197178, 203641, 205758, 211816, 215223, 223434, 223551, 2283

In [81]:
distance_memo(T,79,79)

413238

In [52]:
t0=time()
distance_prog_dyn(T,79,79)
print(time()-t0)

0.013000011444091797


In [53]:
t0=time()
distance_memo(T,79,79)
print(time()-t0)

0.0


In [54]:
t0=time()
#distance(T,79,79)
print(time()-t0)

0.0
